# Name(s):  (Group Assignment - one person turns the notebook in, all members are required to review)

# Date:

# Class:  (Summer 2020)

# Practicing Linear Regression on the US Housing Price Data "USA_Housing.csv"
## Knowledge Discovery in Databases - Data Mining
## DIRECTIONS
### Review all code and markdown.  Provide responses to questions at the end of the notebook (markdown, code)
### Turn in this notebook via Canvas in the assignment area
#### credits:  Github tirthajyoti/Machine-Learning-with-Python
#### prepared w/edits for UNCC by Dr. Pamela Thompson

## PART 1:  Review this primer on Linear Regression

In statistics, linear regression is a popular approach for modeling the relationship between a scalar dependent variable y and one or more explanatory variables (or independent variables) denoted X. The case of one explanatory variable is called simple linear regression. For more than one explanatory variable, the model is labeled multiple linear regression.

In linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Such models are called linear models. Most commonly, the conditional mean of the response given the values of the explanatory variables (or predictors) is assumed to be an affine function of those values; less commonly, the conditional median or some other quantile is used. Like all forms of regression analysis, linear regression focuses on the conditional probability distribution of the response given the values of the predictors, rather than on the joint probability distribution of all of these variables, which is the domain of multivariate analysis. 

Linear regression models are often fitted using the least squares approach, but they may also be fitted in other ways (see note at end). Additionally, the least squares approach can be used to fit models that are not linear models. Although the terms "least squares" and "linear model" are closely linked, they are not synonymous.

NOTE:  Other modelings for linear regression include minimizing the "lack of fit" in some other norm (as with least absolute deviations regression), or by minimizing a penalized version of the least squares loss function as in ridge regression ($L_2$-norm penalty) and lasso ($L_1$-norm penalty).

### A.  Import packages and dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
df = pd.read_csv("USA_Housing.csv")
df.head()

### B.  Check basic information on the data set - our ulitmate goal will be to predict price based on the other numeric variables such as average area income, average area house age, etc.
### Note price is a numeric variable (linear regression is suitable to predict price as a supervised method)

In [ ]:
#info to get basic information
df.info(verbose=True)

In [ ]:
#describe() to get the statistical summary of the various features of the data set
df.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

**'columns' method to get the names of the columns (features)**

In [ ]:
df.columns

### Let's do some exploratory data analysis with basic plotting and visualization on the data set

**Pairplots using seaborn**

In [ ]:
sns.pairplot(df)

**Distribution of price (the predicted quantity)**

In [ ]:
df['Price'].plot.hist(bins=25,figsize=(8,4))

In [ ]:
df['Price'].plot.density()

**Correlation matrix and heatmap**

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df.corr(),annot=True,linewidths=2)

### Feature and variable sets

**Make a list of data frame column names**

In [ ]:
l_column = list(df.columns) # Making a list out of column names
len_feature = len(l_column) # Length of column vector list
l_column

**Put all the numerical features in X and Price in y, ignore Address which is string and of course will not be used for our model for linear regression**

In [ ]:
X = df[l_column[0:len_feature-2]]
y = df[l_column[len_feature-2]]

In [ ]:
print("Feature set size:",X.shape)
print("Variable set size:",y.shape)

In [ ]:
X.head()

In [ ]:
y.head()

### Test-train split
#### Now we will split our data into a training and testing split - 70% training and 30% testing.  We will train the model on the training split then evaluate the model on the test split.

**Import train_test_split function from scikit-learn**

In [ ]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

**Create X and y train and test splits in one command using a split ratio and a random seed**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

**Check the size and shape of train/test splits (it should be in the ratio as per test_size parameter above)**

In [ ]:
print("Training feature set size:",X_train.shape)
print("Test feature set size:",X_test.shape)
print("Training variable set size:",y_train.shape)
print("Test variable set size:",y_test.shape)

### Model fit and training

**Import linear regression model estimator from scikit-learn and instantiate**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
lm = LinearRegression() # Creating a Linear Regression object 'lm'

**Fit the model on to the instantiated object itself**

In [ ]:
lm.fit(X_train,y_train) # Fit the linear model on to the 'lm' object itself i.e. no need to set this to another variable

**Check the intercept and coefficients and put them in a DataFrame**

In [ ]:
print("The intercept term of the linear model:", lm.intercept_)

In [ ]:
print("The coefficients of the linear model:", lm.coef_)

In [ ]:
#idict = {'Coefficients':lm.intercept_}
#idf = pd.DataFrame(data=idict,index=['Intercept'])
cdf = pd.DataFrame(data=lm.coef_, index=X_train.columns, columns=["Coefficients"])
#cdf=pd.concat([idf,cdf], axis=0)
cdf

### Calculation of standard errors and t-statistic for the coefficients
### Important:  see list of features in the order of importance for predicting the house price

In [ ]:
n=X_train.shape[0]
k=X_train.shape[1]
dfN = n-k
train_pred=lm.predict(X_train)
train_error = np.square(train_pred - y_train)
sum_error=np.sum(train_error)
se=[0,0,0,0,0]
for i in range(k):
    r = (sum_error/dfN)
    r = r/np.sum(np.square(X_train[list(X_train.columns)[i]]-X_train[list(X_train.columns)[i]].mean()))
    se[i]=np.sqrt(r)
cdf['Standard Error']=se
cdf['t-statistic']=cdf['Coefficients']/cdf['Standard Error']
cdf

In [ ]:
print("Therefore, features arranged in the order of importance for predicting the house price\n",'-'*90,sep='')
l=list(cdf.sort_values('t-statistic',ascending=False).index)
print(' > \n'.join(l))

In [ ]:
l=list(cdf.index)
from matplotlib import gridspec
fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(2,3)
#f, ax = plt.subplots(nrows=1,ncols=len(l), sharey=True)
ax0 = plt.subplot(gs[0])
ax0.scatter(df[l[0]],df['Price'])
ax0.set_title(l[0]+" vs. Price", fontdict={'fontsize':20})

ax1 = plt.subplot(gs[1])
ax1.scatter(df[l[1]],df['Price'])
ax1.set_title(l[1]+" vs. Price",fontdict={'fontsize':20})

ax2 = plt.subplot(gs[2])
ax2.scatter(df[l[2]],df['Price'])
ax2.set_title(l[2]+" vs. Price",fontdict={'fontsize':20})

ax3 = plt.subplot(gs[3])
ax3.scatter(df[l[3]],df['Price'])
ax3.set_title(l[3]+" vs. Price",fontdict={'fontsize':20})

ax4 = plt.subplot(gs[4])
ax4.scatter(df[l[4]],df['Price'])
ax4.set_title(l[4]+" vs. Price",fontdict={'fontsize':20})

**R-square of the model fit**

In [ ]:
print("R-squared value of this fit:",round(metrics.r2_score(y_train,train_pred),3))

### Prediction, error estimate, and regression evaluation matrices

**Prediction using the lm model**

In [ ]:
predictions = lm.predict(X_test)
print ("Type of the predicted object:", type(predictions))
print ("Size of the predicted object:", predictions.shape)

**Scatter plot of predicted price and y_test set to see if the data fall on a 45 degree straight line**

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Actual vs. predicted house prices",fontsize=25)
plt.xlabel("Actual test set house prices",fontsize=18, color="red")
plt.ylabel("Predicted house prices", fontsize=18)
plt.scatter(x=y_test,y=predictions)

**Plotting histogram of the residuals i.e. predicted errors (expect a normally distributed pattern)**

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Histogram of residuals to check for normality",fontsize=25)
plt.xlabel("Residuals",fontsize=18)
plt.ylabel("Kernel density", fontsize=18)
sns.distplot([y_test-predictions])

**Scatter plot of residuals and predicted values (Homoscedasticity)**<p>Why do we care?</p><p><a href="https://www.statisticshowto.datasciencecentral.com/heteroscedasticity-simple-definition-examples/">Heteroscedasticity and Homoscedasticity</a></p>
### This is important!  Take the time to review the article!

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Residuals vs. predicted values plot (Homoscedasticity)\n",fontsize=25)
plt.xlabel("Predicted house prices",fontsize=18)
plt.ylabel("Residuals", fontsize=18)
plt.scatter(x=predictions,y=y_test-predictions)

**Regression evaluation metrices**

In [ ]:
print("Mean absolute error (MAE):", metrics.mean_absolute_error(y_test,predictions))
print("Mean square error (MSE):", metrics.mean_squared_error(y_test,predictions))
print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,predictions)))

**R-square value** - the R-sqared value of predictions is .919
<p>Evaluating R-squared - the coefficient of Determination</p><p><a href="https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/coefficient-of-determination-r-squared/">R-squared</a></p>

In [ ]:
print("R-squared value of predictions:",round(metrics.r2_score(y_test,predictions),3))

QUESTIONS - ANSWER THE FOLLOWING QUESTIONS BY INSERTING MARKDOWN CELLS BELOW WITH THE QUESTION NUMBER AND THE ANSWER

1.  What type of variables are acceptable for linear regression for the X variable(s)?  The Y variable?<br>
2.  What variable was discarded from the original dataset and why?
3.  What is the purpose of the training and test data set?
4.  When do you need to standardize the variables in regression analysis?
5.  What is the purpose of plotting the variables to check for homoscedasticity?
6.  This model produced a coefficient of determination of .919.  Is this good?  What does it mean?

1.

2.

3.

4.

5.

6.